# Movie Recommendations


Load the necessary libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

## Preprocessing data
Load the data

In [3]:
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')

## Preprocess Movies Data

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_df['genres'] = movies_df.apply(lambda row: row.genres.split('|'), axis = 1)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


## Make list of genres

In [6]:
genres = "Action* Adventure* Animation* Children's* Comedy* Crime* Documentary* Drama* Fantasy* Film-Noir* Horror* Musical* Mystery* Romance* Sci-Fi* Thriller* War* Western* (no genres listed)".split("* ")
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 '(no genres listed)']

## Check ratings

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
ratings_df = ratings_df.merge(movies_df, how = 'left', on = 'movieId')
ratings_df.head(15)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,3,4.0,964981247,Grumpier Old Men (1995),"[Comedy, Romance]"
2,1,6,4.0,964982224,Heat (1995),"[Action, Crime, Thriller]"
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,5.0,964982931,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"
5,1,70,3.0,964982400,From Dusk Till Dawn (1996),"[Action, Comedy, Horror, Thriller]"
6,1,101,5.0,964980868,Bottle Rocket (1996),"[Adventure, Comedy, Crime, Romance]"
7,1,110,4.0,964982176,Braveheart (1995),"[Action, Drama, War]"
8,1,151,5.0,964984041,Rob Roy (1995),"[Action, Drama, Romance, War]"
9,1,157,5.0,964984100,Canadian Bacon (1995),"[Comedy, War]"


# Knowledge based Recommender System
This is used if we have no user data. This is based on popularity

In [9]:
avg_rated_movies = ratings_df.groupby(['title']).agg({'rating': 'mean'})['rating'].sort_values(ascending=False)
avg_rated_movies

title
Gena the Crocodile (1969)                  5.0
True Stories (1986)                        5.0
Cosmic Scrat-tastrophe (2015)              5.0
Love and Pigeons (1985)                    5.0
Red Sorghum (Hong gao liang) (1987)        5.0
                                          ... 
Don't Look Now (1973)                      0.5
Journey 2: The Mysterious Island (2012)    0.5
Joe Dirt 2: Beautiful Loser (2015)         0.5
Jesus Christ Vampire Hunter (2001)         0.5
Fullmetal Alchemist 2018 (2017)            0.5
Name: rating, Length: 9719, dtype: float64

In [10]:
popular_movies = ratings_df.groupby(['title']).agg({"rating":"count"})['rating'].sort_values(ascending=False)
popular_movies

title
Forrest Gump (1994)                          329
Shawshank Redemption, The (1994)             317
Pulp Fiction (1994)                          307
Silence of the Lambs, The (1991)             279
Matrix, The (1999)                           278
                                            ... 
King Solomon's Mines (1950)                    1
King Solomon's Mines (1937)                    1
King Ralph (1991)                              1
King Kong Lives (1986)                         1
À nous la liberté (Freedom for Us) (1931)      1
Name: rating, Length: 9719, dtype: int64

In [11]:
popular_movies = popular_movies.where(popular_movies > 10).dropna()
popular_movies.head(15)

title
Forrest Gump (1994)                                      329.0
Shawshank Redemption, The (1994)                         317.0
Pulp Fiction (1994)                                      307.0
Silence of the Lambs, The (1991)                         279.0
Matrix, The (1999)                                       278.0
Star Wars: Episode IV - A New Hope (1977)                251.0
Jurassic Park (1993)                                     238.0
Braveheart (1995)                                        237.0
Terminator 2: Judgment Day (1991)                        224.0
Schindler's List (1993)                                  220.0
Fight Club (1999)                                        218.0
Toy Story (1995)                                         215.0
Star Wars: Episode V - The Empire Strikes Back (1980)    211.0
Usual Suspects, The (1995)                               204.0
American Beauty (1999)                                   204.0
Name: rating, dtype: float64

In [13]:
popularity_recommendations = pd.merge(popular_movies, avg_rated_movies, how = 'inner', on = 'title')
popularity_recommendations.head(15)

,rating_x,rating_y
title,,
Forrest Gump (1994),329.0,4.164134
"Shawshank Redemption, The (1994)",317.0,4.429022
Pulp Fiction (1994),307.0,4.197068
"Silence of the Lambs, The (1991)",279.0,4.161290
"Matrix, The (1999)",278.0,4.192446
Star Wars: Episode IV - A New Hope (1977),251.0,4.231076
Jurassic Park (1993),238.0,3.750000
Braveheart (1995),237.0,4.031646
Terminator 2: Judgment Day (1991),224.0,3.970982


In [14]:
popularity_recommendations.rename({'rating_x': 'Number of users', 'rating_y': 'Average rating'}, axis = 1, inplace = True)
popularity_recommendations.sort_values(by = 'Average rating', ascending = False, inplace = True)
popularity_recommendations.head(10)

,Number of users,Average rating
title,,
Secrets & Lies (1996),11.0,4.590909
Guess Who's Coming to Dinner (1967),11.0,4.545455
Paths of Glory (1957),12.0,4.541667
"Streetcar Named Desire, A (1951)",20.0,4.475000
"Celebration, The (Festen) (1998)",12.0,4.458333
Ran (1985),15.0,4.433333
"Shawshank Redemption, The (1994)",317.0,4.429022
His Girl Friday (1940),14.0,4.392857
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),27.0,4.333333


## Recommending based on Genre
sorting by genre

In [15]:
genre_recommendations = popularity_recommendations.merge(movies_df, how = 'left', on = 'title')
genre_recommendations

,title,Number of users,Average rating,movieId,genres
0,Secrets & Lies (1996),11.0,4.590909,1041,[Drama]
1,Guess Who's Coming to Dinner (1967),11.0,4.545455,3451,[Drama]
2,Paths of Glory (1957),12.0,4.541667,1178,"[Drama, War]"
3,"Streetcar Named Desire, A (1951)",20.0,4.475000,1104,[Drama]
4,"Celebration, The (Festen) (1998)",12.0,4.458333,2360,[Drama]
...,...,...,...,...,...
2119,Superman IV: The Quest for Peace (1987),16.0,1.687500,2643,"[Action, Adventure, Sci-Fi]"
2120,Battlefield Earth (2000),19.0,1.657895,3593,"[Action, Sci-Fi]"
2121,"Flintstones in Viva Rock Vegas, The (2000)",12.0,1.625000,3564,"[Children, Comedy]"
2122,Speed 2: Cruise Control (1997),19.0,1.605263,1556,"[Action, Romance, Thriller]"


In [16]:
genre_recommendations.drop('movieId', inplace = True, axis = 1)
genre_recommendations.head(15)

,title,Number of users,Average rating,genres
0,Secrets & Lies (1996),11.0,4.590909,[Drama]
1,Guess Who's Coming to Dinner (1967),11.0,4.545455,[Drama]
2,Paths of Glory (1957),12.0,4.541667,"[Drama, War]"
3,"Streetcar Named Desire, A (1951)",20.0,4.475000,[Drama]
4,"Celebration, The (Festen) (1998)",12.0,4.458333,[Drama]
5,Ran (1985),15.0,4.433333,"[Drama, War]"
6,"Shawshank Redemption, The (1994)",317.0,4.429022,"[Crime, Drama]"
7,His Girl Friday (1940),14.0,4.392857,"[Comedy, Romance]"
8,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),27.0,4.333333,"[Drama, Film-Noir, Romance]"
9,"Hustler, The (1961)",18.0,4.333333,[Drama]


In [17]:
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 '(no genres listed)']

In [18]:
# genre_recommendations['Year Released'] = genre_recommendations.apply(lambda row: row.genres.split('|'), axis = 1)

In [19]:
genre_recommendations[genre_recommendations['genres'].apply(lambda x: 'Adventure' in x)].head(15)

,title,Number of users,Average rating,genres
17,Lawrence of Arabia (1962),45.0,4.300000,"[Adventure, Drama, War]"
35,"Outlaw Josey Wales, The (1976)",18.0,4.250000,"[Action, Adventure, Drama, Thriller, Western]"
39,"Princess Bride, The (1987)",142.0,4.232394,"[Action, Adventure, Comedy, Fantasy, Romance]"
40,Star Wars: Episode IV - A New Hope (1977),251.0,4.231076,"[Action, Adventure, Sci-Fi]"
41,Yojimbo (1961),13.0,4.230769,"[Action, Adventure]"
53,Star Wars: Episode V - The Empire Strikes Back...,211.0,4.215640,"[Action, Adventure, Sci-Fi]"
58,Raiders of the Lost Ark (Indiana Jones and the...,200.0,4.207500,"[Action, Adventure]"
68,Seven Samurai (Shichinin no samurai) (1954),48.0,4.187500,"[Action, Adventure, Drama]"
69,North by Northwest (1959),57.0,4.184211,"[Action, Adventure, Mystery, Romance, Thriller]"
86,Monty Python and the Holy Grail (1975),136.0,4.161765,"[Adventure, Comedy, Fantasy]"


# Recommendations based on User (KNN)
This will be taking into account what movies the user has watched and will recommend movies based on what similar users have enjoyed watching.

In [20]:
ratings_df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,3,4.0,964981247,Grumpier Old Men (1995),"[Comedy, Romance]"
2,1,6,4.0,964982224,Heat (1995),"[Action, Crime, Thriller]"
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,5.0,964982931,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),"[Drama, Horror, Thriller]"
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),"[Action, Crime, Thriller]"
100833,610,168250,5.0,1494273047,Get Out (2017),[Horror]
100834,610,168252,5.0,1493846352,Logan (2017),"[Action, Sci-Fi]"


In [39]:
refined_df = ratings_df.groupby(by=['userId', 'title'], as_index = False).agg({'rating': 'mean'})
refined_df.head(15)

,userId,title,rating
0,1,"13th Warrior, The (1999)",4.0
1,1,20 Dates (1998),4.0
2,1,"Abyss, The (1989)",4.0
3,1,"Adventures of Robin Hood, The (1938)",5.0
4,1,Alice in Wonderland (1951),5.0
5,1,Alien (1979),4.0
6,1,All Quiet on the Western Front (1930),5.0
7,1,American Beauty (1999),5.0
8,1,American History X (1998),5.0
9,1,"American Tail, An (1986)",5.0


In [40]:
user_to_movie_df = refined_df.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
user_to_movie_df.head(15)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [23]:
user_to_movie_sparse_df = csr_matrix(user_to_movie_df.values)
knn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
knn_model.fit(user_to_movie_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [24]:
def get_similar_users(user, n = 5):
    knn_input = np.asarray([user_to_movie_df.values[user-1]])
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
    
    print(f"Top {n} users who are very similar to User-{user} are: ")
    for i in range(1,len(distances[0])):
        print(i,". User:", indices[0][i]+1, "separated by distance of",distances[0][i])
    return indices.flatten()[1:] + 1, distances.flatten()[1:]

In [25]:
user = 5

similar_user_list, distance_list = get_similar_users(user)

Top 5 users who are very similar to User-5 are: 
1 . User: 470 separated by distance of 0.4793292042473569
2 . User: 229 separated by distance of 0.48266710413766245
3 . User: 565 separated by distance of 0.4886746437839188
4 . User: 235 separated by distance of 0.4923618395241436
5 . User: 142 separated by distance of 0.5258758000611337


In [26]:
movies_list = user_to_movie_df.columns
movies_list

Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)', '*batteries not included (1987)',
       ...
       'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       '¡Three Amigos! (1986)', 'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=9719)

In [27]:
len(movies_list)

9719

In [28]:
weightage_list = distance_list/np.sum(distance_list)
weightage_list

array([0.19414619, 0.19549817, 0.19793144, 0.1994249 , 0.2129993 ])

In [29]:
mov_rtngs_sim_users = user_to_movie_df.values[similar_user_list]
mov_rtngs_sim_users

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
weightage_list = weightage_list[:,np.newaxis] + np.zeros(len(movies_list))
weightage_list.shape

(5, 9719)

In [31]:
new_rating_matrix = weightage_list*mov_rtngs_sim_users
mean_rating_list = new_rating_matrix.sum(axis =0)
mean_rating_list

array([0., 0., 0., ..., 0., 0., 0.])

In [32]:
def recommend_movies(n):
  n = min(len(mean_rating_list),n)
  print(list(movies_list[np.argsort(mean_rating_list)[::-1][:n]]))

In [33]:
print("Movies recommended based on similar users are: ")
recommend_movies(10)

Movies recommended based on similar users are: 
['Forrest Gump (1994)', 'Finding Nemo (2003)', 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)', "Schindler's List (1993)", 'Pretty Woman (1990)', 'Interview with the Vampire: The Vampire Chronicles (1994)', 'Toy Story 3 (2010)', 'Alien (1979)', 'Shawshank Redemption, The (1994)', 'Twister (1996)']


In [35]:
ratings_df.loc[ratings_df['userId'] == 5].sort_values('rating', ascending = False).head(15)['title']

553                 Dances with Wolves (1990)
546          In the Name of the Father (1993)
548                   Schindler's List (1993)
522         Postman, The (Postino, Il) (1994)
557                          Pinocchio (1940)
556               Beauty and the Beast (1991)
527                 Heavenly Creatures (1994)
555    Snow White and the Seven Dwarfs (1937)
533                       Pulp Fiction (1994)
532                 Once Were Warriors (1994)
516                          Toy Story (1995)
541                          Mask, The (1994)
517                         Get Shorty (1995)
549                 Secret Garden, The (1993)
551                            Aladdin (1992)
Name: title, dtype: object